<a href="https://colab.research.google.com/github/Bhekmuzi/water-usage-norm/blob/main/src/data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip freeze > requirements.txt

In [2]:
# !pip install influxdb

In [3]:
# !pip install pandas pymongo

In [4]:
import numpy as np
from sklearn.cluster import KMeans
# pip install influxdb
# !python --version
import numpy as np
import pandas as pd
import datetime
import time
from influxdb import InfluxDBClient
from pymongo import MongoClient
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
# from scipy.stats import entropy

# Provide the IP address, username, password, database name, RFC3339 standard time format, and create a connection client for the 'db0' database
client = InfluxDBClient('59.120.114.133', 8086, 'telegraf', 'telegraf', 'db0', 'rfc3339', timeout=10)

In [5]:
# Function to calculate start and end times for the previous day
def calculate_previous_day_times():
    current_datetime = datetime.now()
    start_of_previous_day = current_datetime - timedelta(days=1)
    start_of_previous_day = start_of_previous_day.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_previous_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(microseconds=1)

    return start_of_previous_day, end_of_previous_day

In [6]:
# InfluxDB query
start_time, end_time = calculate_previous_day_times()

sql_string = f'SELECT DISTINCT("value") AS value FROM mbMQTT6 WHERE "topic" = \'mbMQTT2/home2127/C2BDF8/TH20\' AND time >= \'{start_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' AND time <= \'{end_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' GROUP BY time(10s) FILL(previous) ORDER BY time ASC TZ(\'Asia/Taipei\');'

result = client.query(sql_string) #

In [7]:

homes_data = {
    'home2127':pd.DataFrame(result['mbMQTT6']),
    'home2128':pd.DataFrame(result['mbMQTT6'])
}

In [8]:
# Function to fill missing values for a given home
def fill_missing_values(home_data):
    home_data['time'] = pd.to_datetime(home_data['time'])
    home_data.set_index('time', inplace=True)
    expected_time_intervals = pd.date_range(start=home_data.index.min(), end=home_data.index.max(), freq='10S')
    home_data = home_data.reindex(expected_time_intervals)
    home_data['value'] = home_data['value'].fillna(method='pad')
    home_data.reset_index(inplace=True)
    return home_data

In [9]:
# Loop through each home and fill missing values
filled_homes_data = {}
for home_name, home_df in homes_data.items():
    filled_homes_data[home_name] = fill_missing_values(home_df)

# # Display the resulting filled DataFrames
# for home_name, filled_home_data in filled_homes_data.items():
#     print(f"\nFilled Data for {home_name}:\n{filled_home_data}")

In [ ]:
# Loop through each home and perform the division and multiplication
transformed_homes_data = {}
for home_name, filled_home_data in filled_homes_data.items():
    # Convert 'value' column to numeric
    filled_home_data['value'] = pd.to_numeric(filled_home_data['value'], errors='coerce')

    # Perform division and multiplication
    filled_home_data['value'] = filled_home_data['value'] / 100000 * 1000

    # Store the transformed DataFrame
    transformed_homes_data[home_name] = filled_home_data

# # Display the resulting transformed DataFrames
# for home_name, transformed_home_data in transformed_homes_data.items():
#     print(f"\nTransformed Data for {home_name}:\n{transformed_home_data}")


In [18]:
# prompt: loop through each home and find the difference between 'value'
differenced_homes_data = {}
for home_name, transformed_home_data in transformed_homes_data.items():
    # Calculate the difference between 'value' column
    transformed_home_data['volume'] = transformed_home_data['value'].diff()

    # Store the transformed DataFrame
    differenced_homes_data[home_name] = transformed_home_data

# # Display the resulting DataFrames with filled NaN values
# for home_name, differenced_home_data in differenced_homes_data.items():
#     print(f"\nData for {home_name} with NaN filled with 0:\n{differenced_home_data}")

In [19]:
# Loop through each home and replace NaN with 0 in the 'value_diff' column
filled_nan_homes_data = {}
for home_name, differenced_home_data in differenced_homes_data.items():
    # Replace NaN with 0 in the 'value_diff' column
    differenced_home_data['volume'] = differenced_home_data['volume'].fillna(0)

    # Store the DataFrame with filled NaN values
    filled_nan_homes_data[home_name] = differenced_home_data

# # Display the resulting DataFrames with filled NaN values
# for home_name, filled_nan_home_data in filled_nan_homes_data.items():
#     print(f"\nData for {home_name} with NaN filled with 0:\n{filled_nan_home_data}")


In [20]:
# Loop through each home and filter values less than 0.2 in the 'value_diff' column
filtered_homes_data = {}
for home_name, filled_nan_home_data in filled_nan_homes_data.items():
    # Filter values less than 0.2 in the 'value_diff' column
    filtered_home_data = filled_nan_home_data[filled_nan_home_data['value_diff'] < 0.2]

    # Store the filtered DataFrame
    filtered_homes_data[home_name] = filtered_home_data

# Display the resulting filtered DataFrames
for home_name, filtered_home_data in filtered_homes_data.items():
    print(f"\nFiltered Data for {home_name}:\n{filtered_home_data}")



Filtered Data for home2127:
                         index     value  value_diff  volume
0    2024-01-03 00:00:00+00:00  26908.90        0.00    0.00
1    2024-01-03 00:00:10+00:00  26908.90        0.00    0.00
2    2024-01-03 00:00:20+00:00  26908.90        0.00    0.00
3    2024-01-03 00:00:30+00:00  26908.90        0.00    0.00
4    2024-01-03 00:00:40+00:00  26908.90        0.00    0.00
...                        ...       ...         ...     ...
8635 2024-01-03 23:59:10+00:00  27073.54        0.00    0.00
8636 2024-01-03 23:59:20+00:00  27073.54        0.00    0.00
8637 2024-01-03 23:59:30+00:00  27073.56        0.02    0.02
8638 2024-01-03 23:59:40+00:00  27073.56        0.00    0.00
8639 2024-01-03 23:59:50+00:00  27073.56        0.00    0.00

[8535 rows x 4 columns]

Filtered Data for home2128:
                         index     value  value_diff  volume
0    2024-01-03 00:00:00+00:00  26908.90        0.00    0.00
1    2024-01-03 00:00:10+00:00  26908.90        0.00    0.00
2 